In [63]:
import time
import pandas as pd
import numpy as np
from io import BytesIO
#specialized imports
import re
import io
import os
import stockfish
from stockfish import Stockfish
import chess
import sys
from tqdm import tqdm
# Create a Stockfish engine instance with AVX2 support
stockfish = Stockfish(r"C:\Desktop\stockfish\stockfish-windows-x86-64-avx2.exe")

In [2]:
games_df = pd.read_csv(r"C:\Users\CDHan\Documents\Online+Chess+Games\chess_games_to_analyze.csv")

In [3]:
print(games_df)

       game_id                                              moves  WhiteMove1  \
0            1  d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...         NaN   
1            2  d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...         NaN   
2            3  e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...         NaN   
3            4  d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...         NaN   
4            5  e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...         NaN   
...        ...                                                ...         ...   
20053    20054  d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...         NaN   
20054    20055  d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...         NaN   
20055    20056  d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...         NaN   
20056    20057  e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...         NaN   
20057    20058  d4 d5 Bf4 Na6 e3 e6 c3 Nf6 Nf3 Bd7 Nbd2 b5 Bd3...         NaN   

       WhiteEvaluation1  Wh

In [4]:
# Splitting first 10 moves to single cells

Move_Columns = ["WhiteMove1", "BlackMove1", "WhiteMove2", "BlackMove2", "WhiteMove3", "BlackMove3",
                "WhiteMove4", "BlackMove4", "WhiteMove5", "BlackMove5", "WhiteMove6", "BlackMove6",
                "WhiteMove7", "BlackMove7", "WhiteMove8", "BlackMove8", "WhiteMove9", "BlackMove9",
                "WhiteMove10", "BlackMove10"]

import chess

# Convert moves to the format expected by Stockfish
def convert_moves(x):
    game_list = x.split(" ")
    moves = game_list[:20]
    stockfish_moves = []
    board = chess.Board()
    for move in moves:
        board.push_san(move)
        uci_move = board.peek().uci()  # Get the UCI representation of the move
        stockfish_moves.append(uci_move)
    return stockfish_moves

games_df[Move_Columns] = games_df['moves'].apply(convert_moves).apply(pd.Series)


In [5]:
#creating a simplified dataframe for games where the same line was played out to move 10
simplified_df = games_df.drop_duplicates(Move_Columns).copy()
#Creating a backup of simplified_df
simplified_copy = simplified_df.copy()


In [6]:
print(simplified_df)

       game_id                                              moves WhiteMove1  \
0            1  d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...       d2d4   
1            2  d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...       d2d4   
2            3  e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...       e2e4   
3            4  d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...       d2d4   
4            5  e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...       e2e4   
...        ...                                                ...        ...   
20053    20054  d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...       d2d4   
20054    20055  d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...       d2d4   
20055    20056  d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...       d2d4   
20056    20057  e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...       e2e4   
20057    20058  d4 d5 Bf4 Na6 e3 e6 c3 Nf6 Nf3 Bd7 Nbd2 b5 Bd3...       d2d4   

       WhiteEvaluation1  WhiteBestMove1

In [92]:
"""This block defines the function that the code will pass over the move columns of the dataframe.
in the next block, we will use it to analyze the first ten moves of the game that was played and, 
for each game move, return the best move and the stockfish evaluation after the move was played."""

def analyze_moves(*moves):
    moves_list = [move for move in moves if move is not None and not isinstance(move, float)]
    #print(moves_list)
    moves_len = len(moves_list)
    result = None
    evaluation = None
    if moves_len > 1:
        stockfish.set_position(moves_list[:-1])  # Set the position before the move was made.
    else:
        stockfish.set_position() #set starting position
    result = stockfish.get_best_move()  # Get the best move from Stockfish
    last_move = moves_list[-1]
    stockfish.set_position(moves_list)  # Make the last move on Stockfish
    evaluation_dict = stockfish.get_evaluation()  # Get the evaluation from Stockfish
    if evaluation_dict and "value" in evaluation_dict:
        evaluation = evaluation_dict["value"]
    else:
        evaluation = None

    return evaluation, result

In [ ]:
""" This block passes over the first ten moves (in chess terms each player's move is a half-move, 
so this means up to twenty moves per row) of the games in the dataframe and places the best move 
for each move in a corresponding column.
Warning: This code may take up to 20 hours to run. additional code which picks up this process where 
it left off is available below."""


#resetting simplified df with the copy
simplified_df=simplified_copy.copy() 
active_columns = []
eval_columns = best_columns = [col for col in simplified_copy.columns if 'Evaluation' in col]
best_columns = [col for col in simplified_copy.columns if 'Best' in col]

try:
    simplified_df = simplified_df.drop(columns=eval_columns+best_columns)
except Exception as e:
    print(e)

for move_column, eval_column, best_column in zip(Move_Columns, eval_columns, best_columns):
    #backing up simplified_df
    active_columns = active_columns + [move_column]
    working_df = simplified_copy.drop_duplicates(active_columns).copy()
    working_df = working_df[active_columns] 
    working_df[[eval_column,best_column]] = None,None
    working_df = working_df.dropna(how='all')
    working_df = working_df.dropna(subset=[move_column])
    #print(working_df)
    # Apply the analyze_moves function to the working_df and update the analysis_results DataFrame
    for index, row in tqdm(working_df.iterrows(), total=len(working_df), desc='Processing rows'):
        try:
            evaluation, best_move = analyze_moves(*row[active_columns])
            #print("Best Move: "+str(best_move)+"\n Evaluation: "+str(evaluation))
            working_df.at[index, eval_column] = evaluation
            working_df.at[index, best_column] = best_move
        except KeyboardInterrupt:
            print("Execution interrupted. Current row:", row)
            simplified_df = pd.merge(simplified_df, working_df, on=active_columns, how='left')
            break
    simplified_df = pd.merge(simplified_df, working_df, on=active_columns, how='left')
    advance = input(move_column + " analyzed.\n Would you like to continue? (y/n)")
    if advance == 'y':
        continue
    else:
        break
print(simplified_df)


Processing rows:  70%|█████████████████████████████████████████████▌                   | 14/20 [00:04<00:02,  2.87it/s]


Execution interrupted. Current row: WhiteMove1          d2d3
WhiteEvaluation1    None
WhiteBestMove1      None
Name: 475, dtype: object


In [74]:
#Moemorializing the analysis
simplified_df.to_csv(r'C:\Users\CDHan\Documents\Online+Chess+Games\simplified_df.csv', index=False)

In [85]:
#Repeating the above/picking up where the code left off. 
pickup_df = pd.read_csv(r'C:\Users\CDHan\Documents\Online+Chess+Games\simplified_df.csv')

In [86]:
print(eval_columns)
print(best_columns)

['WhiteEvaluation4', 'BlackEvaluation4', 'WhiteEvaluation5', 'BlackEvaluation5', 'WhiteEvaluation6', 'BlackEvaluation6', 'WhiteEvaluation7', 'BlackEvaluation7', 'WhiteEvaluation8', 'BlackEvaluation8', 'WhiteEvaluation9', 'BlackEvaluation9', 'WhiteEvaluation10', 'BlackEvaluation10']
['WhiteBestMove4', 'BlackBestMove4', 'WhiteBestMove5', 'BlackBestMove5', 'WhiteBestMove6', 'BlackBestMove6', 'WhiteBestMove7', 'BlackBestMove7', 'WhiteBestMove8', 'BlackBestMove8', 'WhiteBestMove9', 'BlackBestMove9', 'WhiteBestMove10', 'BlackBestMove10']


In [87]:
#resetting simplified df with the copy
active_columns = []
lists_df = pd.DataFrame()
Move_Columns = ["WhiteMove1", "BlackMove1", "WhiteMove2", "BlackMove2", "WhiteMove3", "BlackMove3",
                "WhiteMove4", "BlackMove4", "WhiteMove5", "BlackMove5", "WhiteMove6", "BlackMove6",
                "WhiteMove7", "BlackMove7", "WhiteMove8", "BlackMove8", "WhiteMove9", "BlackMove9",
                "WhiteMove10", "BlackMove10"]
eval_columns = ['WhiteEvaluation1', 'BlackEvaluation1', 'WhiteEvaluation2', 'BlackEvaluation2', 
                'WhiteEvaluation3', 'BlackEvaluation3', 'WhiteEvaluation4', 'BlackEvaluation4', 
                'WhiteEvaluation5', 'BlackEvaluation5', 'WhiteEvaluation6', 'BlackEvaluation6', 
                'WhiteEvaluation7', 'BlackEvaluation7', 'WhiteEvaluation8', 'BlackEvaluation8', 
                'WhiteEvaluation9', 'BlackEvaluation9', 'WhiteEvaluation10', 'BlackEvaluation10']
best_columns = ['WhiteBestMove1', 'BlackBestMove1', 'WhiteBestMove2', 'BlackBestMove2',
                'WhiteBestMove3', 'BlackBestMove3', 'WhiteBestMove4', 'BlackBestMove4',
                'WhiteBestMove5', 'BlackBestMove5', 'WhiteBestMove6', 'BlackBestMove6',
                'WhiteBestMove7', 'BlackBestMove7', 'WhiteBestMove8', 'BlackBestMove8',
                'WhiteBestMove9', 'BlackBestMove9', 'WhiteBestMove10', 'BlackBestMove10']
pickup_df_columns = pickup_df.columns.tolist()

lists_df = pd.DataFrame({
    'Move_Columns': Move_Columns,
    'eval_columns': eval_columns,
    'best_columns': best_columns
})

lists_df['Done?'] = lists_df['best_columns'].isin(pickup_df.columns)

done_df = lists_df[lists_df['Done?']==True]
to_do_df = lists_df[lists_df['Done?']==False]

active_columns = done_df['Move_Columns'].tolist()
to_do_columns = to_do_df['Move_Columns'].tolist()
eval_columns = to_do_df['eval_columns'].tolist()
best_columns = to_do_df['best_columns'].tolist()

In [88]:
#print('Eval Columns')
#print(eval_columns)
#print('Dataframe columns')
print(pickup_df_columns)
print(active_columns)

['game_id', 'moves', 'WhiteMove1', 'BlackMove1', 'WhiteMove2', 'BlackMove2', 'WhiteMove3', 'BlackMove3', 'WhiteMove4', 'BlackMove4', 'WhiteMove5', 'BlackMove5', 'WhiteMove6', 'BlackMove6', 'WhiteMove7', 'BlackMove7', 'WhiteMove8', 'BlackMove8', 'WhiteMove9', 'BlackMove9', 'WhiteMove10', 'BlackMove10', 'WhiteEvaluation1', 'WhiteBestMove1', 'BlackEvaluation1', 'BlackBestMove1', 'WhiteEvaluation2', 'WhiteBestMove2', 'BlackEvaluation2', 'BlackBestMove2', 'WhiteEvaluation3', 'WhiteBestMove3', 'BlackEvaluation3', 'BlackBestMove3']
['WhiteMove1', 'BlackMove1', 'WhiteMove2', 'BlackMove2', 'WhiteMove3', 'BlackMove3']


In [89]:
for move_column, eval_column, best_column in zip(to_do_columns, eval_columns, best_columns):
    #backing up simplified_df
    active_columns = active_columns + [move_column]
    working_df = pickup_df.drop_duplicates(active_columns).copy()
    working_df = working_df[active_columns] 
    working_df[[eval_column,best_column]] = None,None
    working_df = working_df.dropna(how='all')
    working_df = working_df.dropna(subset=[move_column])
    #print(working_df)
    # Apply the analyze_moves function to the working_df and update the analysis_results DataFrame
    for index, row in tqdm(working_df.iterrows(), total=len(working_df), desc='Processing rows'):
        try:
            evaluation, best_move = analyze_moves(*row[active_columns])
             #print("Best Move: "+str(best_move)+"\n Evaluation: "+str(evaluation))
            working_df.at[index, eval_column] = evaluation
            working_df.at[index, best_column] = best_move
        except KeyboardInterrupt:
            print("Execution interrupted. Current row:", row)
            pickup_df = pd.merge(pickup_df, working_df, on=active_columns, how='left')
            break
    pickup_df = pd.merge(pickup_df, working_df, on=active_columns, how='left')
    #advance = input(move_column + " analyzed.\n Would you like to continue? (y/n)")
    #start_time = time.time()

    #while True:
        #advance = input(move_column + " analyzed.\n Would you like to continue? (y/n)")
    
        #if advance == 'y':
            #continue
        #else:
            #break
    
        #elapsed_time = time.time() - start_time
    
        #if elapsed_time > 300:  # 300 seconds = 5 minutes
            #print("No input received within 5 minutes. Continuing...")
            #continue

Processing rows: 100%|█████████████████████████████████████████████████████████| 17538/17538 [1:03:23<00:00,  4.61it/s]


In [90]:
print(pickup_df.columns)

Index(['game_id', 'moves', 'WhiteMove1', 'BlackMove1', 'WhiteMove2',
       'BlackMove2', 'WhiteMove3', 'BlackMove3', 'WhiteMove4', 'BlackMove4',
       'WhiteMove5', 'BlackMove5', 'WhiteMove6', 'BlackMove6', 'WhiteMove7',
       'BlackMove7', 'WhiteMove8', 'BlackMove8', 'WhiteMove9', 'BlackMove9',
       'WhiteMove10', 'BlackMove10', 'WhiteEvaluation1', 'WhiteBestMove1',
       'BlackEvaluation1', 'BlackBestMove1', 'WhiteEvaluation2',
       'WhiteBestMove2', 'BlackEvaluation2', 'BlackBestMove2',
       'WhiteEvaluation3', 'WhiteBestMove3', 'BlackEvaluation3',
       'BlackBestMove3', 'WhiteEvaluation4', 'WhiteBestMove4',
       'BlackEvaluation4', 'BlackBestMove4', 'WhiteEvaluation5',
       'WhiteBestMove5', 'BlackEvaluation5', 'BlackBestMove5',
       'WhiteEvaluation6', 'WhiteBestMove6', 'BlackEvaluation6',
       'BlackBestMove6', 'WhiteEvaluation7', 'WhiteBestMove7',
       'BlackEvaluation7', 'BlackBestMove7', 'WhiteEvaluation8',
       'WhiteBestMove8', 'BlackEvaluation8', 'Bl

In [91]:
pickup_df.to_csv(r'C:\Users\CDHan\Documents\Online+Chess+Games\simplified_df.csv', index=False)